In [1]:
# =========================================================================
# FINE-TUNING - PREPARACIÓN
# =========================================================================

import torch
import pandas as pd

print("="*70)
print("PREPARACIÓN PARA FINE-TUNING")
print("="*70 + "\n")

# Verificar GPU
print("1. VERIFICACIÓN DE GPU")
print("-"*70)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print("Estado: Listo para fine-tuning")
else:
    print("ADVERTENCIA: No hay GPU disponible")
    print("El fine-tuning en CPU tardará 8-12 horas")

# Cargar datos (solo si no están cargados)
print("\n2. CARGA DE DATOS")
print("-"*70)

try:
    # Verificar si df_filtrado ya existe
    print(f"Datos ya cargados: {len(df_filtrado):,} registros")
except NameError:
    # Si no existe, cargar
    print("Cargando datos del CSV...")
    df = pd.read_csv("base_datos_final/base_datos_completa.csv", low_memory=False)
    tipos_relevantes = ['JUNTA_ACCIONISTAS', 'DISOLUCION', 'REMATE']
    df_filtrado = df[df['tipo'].isin(tipos_relevantes)].copy()
    print(f"Datos cargados: {len(df_filtrado):,} registros")

print(f"\nDistribución por tipo:")
for tipo in ['DISOLUCION', 'REMATE', 'JUNTA_ACCIONISTAS']:
    count = len(df_filtrado[df_filtrado['tipo'] == tipo])
    print(f"  {tipo}: {count:,}")

print("\n" + "="*70)
print("LISTO PARA CREAR DATASET")
print("="*70)


PREPARACIÓN PARA FINE-TUNING

1. VERIFICACIÓN DE GPU
----------------------------------------------------------------------
PyTorch version: 2.3.1
CUDA disponible: True
GPU detectada: NVIDIA GeForce RTX 3050 Laptop GPU
Memoria GPU: 4.0 GB
Estado: Listo para fine-tuning

2. CARGA DE DATOS
----------------------------------------------------------------------
Cargando datos del CSV...
Datos cargados: 17,008 registros

Distribución por tipo:
  DISOLUCION: 3,689
  REMATE: 9,515
  JUNTA_ACCIONISTAS: 3,804

LISTO PARA CREAR DATASET


In [2]:
# =========================================================================
# FINE-TUNING - CREACIÓN DEL DATASET
# Metodología basada en Rangan & Yin (2024) y González & Santa (2024)
# =========================================================================

print("="*70)
print("CREACIÓN DE DATASET PARA FINE-TUNING")
print("Metodología: Rangan & Yin (2024) - RAG enhanced fine-tuning")
print("="*70 + "\n")

# Verificar datos disponibles
print(f"Datos disponibles: {len(df_filtrado):,} registros")
print(f"Tipos: {df_filtrado['tipo'].value_counts().to_dict()}\n")

CREACIÓN DE DATASET PARA FINE-TUNING
Metodología: Rangan & Yin (2024) - RAG enhanced fine-tuning

Datos disponibles: 17,008 registros
Tipos: {'REMATE': 9515, 'JUNTA_ACCIONISTAS': 3804, 'DISOLUCION': 3689}



In [3]:
# =========================================================================
# TIPO 1: EXTRACCIÓN DE INFORMACIÓN (40% del dataset)
# =========================================================================

print("Generando ejemplos Tipo 1: Extracción de información...")

ejemplos_extraccion = []

# Tomar muestra balanceada
for tipo in ['DISOLUCION', 'REMATE', 'JUNTA_ACCIONISTAS']:
    df_tipo = df_filtrado[df_filtrado['tipo'] == tipo]
    muestra = df_tipo.sample(min(200, len(df_tipo)), random_state=42)
    
    for idx, row in muestra.iterrows():
        texto = str(row['texto_completo'])[:800]
        
        if tipo == 'DISOLUCION':
            # Construir output esperado
            ruc_clean = str(row.get('ruc', 'No especificado'))
            if ruc_clean == 'nan':
                ruc_clean = 'No especificado'
            
            output = f"""Información extraída del aviso de disolución:

Tipo de documento: Disolución y Liquidación
Empresa: {row.get('empresa', 'No especificado')}
RUC: {ruc_clean}
Fecha de publicación: {row.get('fecha', 'No especificado')}

Fuente: Diario Oficial El Peruano
"""
            
            ejemplo = {
                "instruction": "Extrae la información estructurada del siguiente aviso legal de disolución publicado en el Diario El Peruano. Identifica: tipo de documento, empresa, RUC y fecha.",
                "input": texto,
                "output": output
            }
            
        elif tipo == 'REMATE':
            output = f"""Información extraída del aviso de remate:

Tipo de documento: Remate Judicial
Ubicación del inmueble: {row.get('ubicacion', 'No especificado')}
Valor base: {row.get('base', 'No especificado')}
Fecha de remate: {row.get('fecha', 'No especificado')}

Fuente: Diario Oficial El Peruano
"""
            
            ejemplo = {
                "instruction": "Extrae la información estructurada del siguiente aviso de remate judicial. Identifica: ubicación, valor base y fecha.",
                "input": texto,
                "output": output
            }
            
        else:  # JUNTA_ACCIONISTAS
            ruc_clean = str(row.get('ruc', 'No especificado'))
            if ruc_clean == 'nan':
                ruc_clean = 'No especificado'
            
            output = f"""Información extraída del aviso de junta:

Tipo de documento: Convocatoria a Junta General de Accionistas
Empresa: {row.get('empresa', 'No especificado')}
RUC: {ruc_clean}
Fecha de junta: {row.get('fecha', 'No especificado')}

Fuente: Diario Oficial El Peruano
"""
            
            ejemplo = {
                "instruction": "Extrae la información estructurada del siguiente aviso de convocatoria a junta de accionistas. Identifica: empresa, RUC y fecha.",
                "input": texto,
                "output": output
            }
        
        ejemplos_extraccion.append(ejemplo)

print(f"Ejemplos de extracción creados: {len(ejemplos_extraccion)}")

Generando ejemplos Tipo 1: Extracción de información...
Ejemplos de extracción creados: 600


In [4]:
# =========================================================================
# TIPO 2: FORMATEO PROFESIONAL (30% del dataset)
# =========================================================================

print("\nGenerando ejemplos Tipo 2: Formateo profesional...")

ejemplos_formateo = []

for tipo in ['DISOLUCION', 'REMATE', 'JUNTA_ACCIONISTAS']:
    df_tipo = df_filtrado[df_filtrado['tipo'] == tipo]
    muestra = df_tipo.sample(min(150, len(df_tipo)), random_state=43)
    
    for idx, row in muestra.iterrows():
        # Input: datos desordenados (simular query real)
        if tipo == 'DISOLUCION':
            input_text = f"Empresa {row.get('empresa', 'N/A')} disuelta, RUC {row.get('ruc', 'N/A')}, fecha {row.get('fecha', 'N/A')}"
            
            output_text = f"""AVISO DE DISOLUCIÓN Y LIQUIDACIÓN

De conformidad con el artículo 412 de la Ley General de Sociedades, se comunica:

Empresa: {row.get('empresa', 'No especificado')}
RUC: {row.get('ruc', 'No especificado')}
Fecha de disolución: {row.get('fecha', 'No especificado')}

Publicado en el Diario Oficial El Peruano conforme a ley.
"""
        
        elif tipo == 'REMATE':
            input_text = f"Remate ubicación {row.get('ubicacion', 'N/A')}, base {row.get('base', 'N/A')}, fecha {row.get('fecha', 'N/A')}"
            
            output_text = f"""AVISO DE REMATE JUDICIAL

Por disposición judicial, se convoca a remate público:

Bien inmueble ubicado en: {row.get('ubicacion', 'No especificado')}
Valor base: {row.get('base', 'No especificado')}
Fecha de remate: {row.get('fecha', 'No especificado')}

Publicado en el Diario Oficial El Peruano.
"""
        
        else:  # JUNTA
            input_text = f"Junta de {row.get('empresa', 'N/A')}, fecha {row.get('fecha', 'N/A')}"
            
            output_text = f"""CONVOCATORIA A JUNTA GENERAL DE ACCIONISTAS

Se convoca a los señores accionistas de:

Empresa: {row.get('empresa', 'No especificado')}
RUC: {row.get('ruc', 'No especificado')}
Fecha de junta: {row.get('fecha', 'No especificado')}

Publicado conforme al artículo 43 de la Ley General de Sociedades.
"""
        
        ejemplo = {
            "instruction": "Formatea la siguiente información en un aviso legal profesional, siguiendo la estructura estándar del Diario El Peruano.",
            "input": input_text,
            "output": output_text
        }
        
        ejemplos_formateo.append(ejemplo)

print(f"Ejemplos de formateo creados: {len(ejemplos_formateo)}")



Generando ejemplos Tipo 2: Formateo profesional...
Ejemplos de formateo creados: 450


In [5]:
# =========================================================================
# TIPO 3: PREGUNTA-RESPUESTA CON CONTEXTO (30% del dataset)
# =========================================================================
import random

print("\nGenerando ejemplos Tipo 3: Pregunta-respuesta...")

ejemplos_qa = []

# Plantillas de preguntas
plantillas_preguntas = {
    'DISOLUCION': [
        "¿Cuál es el nombre de la empresa disuelta?",
        "¿Qué RUC tiene la empresa?",
        "¿En qué fecha se publicó la disolución?",
        "¿Qué tipo de documento legal es este?"
    ],
    'REMATE': [
        "¿Dónde está ubicado el inmueble?",
        "¿Cuál es el valor base del remate?",
        "¿Cuándo se realizará el remate?",
        "¿Qué tipo de aviso es este?"
    ],
    'JUNTA_ACCIONISTAS': [
        "¿Qué empresa convoca a junta?",
        "¿Cuándo se realizará la junta?",
        "¿Cuál es el RUC de la empresa?",
        "¿Qué tipo de reunión se convoca?"
    ]
}

for tipo in ['DISOLUCION', 'REMATE', 'JUNTA_ACCIONISTAS']:
    df_tipo = df_filtrado[df_filtrado['tipo'] == tipo]
    muestra = df_tipo.sample(min(150, len(df_tipo)), random_state=44)
    
    for idx, row in muestra.iterrows():
        texto_contexto = str(row['texto_completo'])[:600]
        pregunta = random.choice(plantillas_preguntas[tipo])
        
        # Generar respuesta según pregunta
        if "nombre" in pregunta.lower() or "empresa" in pregunta.lower():
            respuesta = f"La empresa es {row.get('empresa', 'no se especifica en el documento')}."
        elif "ruc" in pregunta.lower():
            ruc_val = row.get('ruc', 'nan')
            if str(ruc_val) == 'nan':
                respuesta = "El RUC no se especifica en el documento."
            else:
                respuesta = f"El RUC de la empresa es {ruc_val}."
        elif "fecha" in pregunta.lower() or "cuándo" in pregunta.lower():
            respuesta = f"La fecha es {row.get('fecha', 'no se especifica en el documento')}."
        elif "ubicado" in pregunta.lower() or "ubicación" in pregunta.lower():
            respuesta = f"El inmueble está ubicado en {row.get('ubicacion', 'no se especifica en el documento')}."
        elif "valor base" in pregunta.lower():
            respuesta = f"El valor base es {row.get('base', 'no se especifica en el documento')}."
        elif "tipo" in pregunta.lower():
            if tipo == 'DISOLUCION':
                respuesta = "Este es un aviso de disolución y liquidación de sociedad."
            elif tipo == 'REMATE':
                respuesta = "Este es un aviso de remate judicial."
            else:
                respuesta = "Esta es una convocatoria a junta general de accionistas."
        else:
            respuesta = "La información solicitada no se encuentra claramente especificada en el documento."
        
        ejemplo = {
            "instruction": f"Lee el siguiente documento legal y responde la pregunta de manera precisa y concisa.\n\nDocumento:\n{texto_contexto}\n\nPregunta: {pregunta}",
            "input": "",
            "output": respuesta
        }
        
        ejemplos_qa.append(ejemplo)

print(f"Ejemplos de QA creados: {len(ejemplos_qa)}")


Generando ejemplos Tipo 3: Pregunta-respuesta...
Ejemplos de QA creados: 450


In [6]:
# =========================================================================
# COMBINAR Y MEZCLAR DATASET
# =========================================================================
import json

from pathlib import Path

print("\n" + "="*70)
print("COMBINANDO DATASET FINAL")
print("="*70 + "\n")

# Combinar todos los ejemplos
dataset_completo = ejemplos_extraccion + ejemplos_formateo + ejemplos_qa

# Mezclar aleatoriamente
random.seed(42)
random.shuffle(dataset_completo)

# Estadísticas
total = len(dataset_completo)
print(f"Total de ejemplos: {total}")
print(f"  Extracción: {len(ejemplos_extraccion)} ({len(ejemplos_extraccion)/total*100:.1f}%)")
print(f"  Formateo: {len(ejemplos_formateo)} ({len(ejemplos_formateo)/total*100:.1f}%)")
print(f"  QA: {len(ejemplos_qa)} ({len(ejemplos_qa)/total*100:.1f}%)")

# Split train/validation (90/10)
split_idx = int(len(dataset_completo) * 0.9)
dataset_train = dataset_completo[:split_idx]
dataset_val = dataset_completo[split_idx:]

print(f"\nSplit:")
print(f"  Train: {len(dataset_train)} ejemplos (90%)")
print(f"  Validation: {len(dataset_val)} ejemplos (10%)")

# Guardar
output_dir = Path("fine_tuning_data")
output_dir.mkdir(exist_ok=True)

with open(output_dir / "dataset_train.json", 'w', encoding='utf-8') as f:
    json.dump(dataset_train, f, ensure_ascii=False, indent=2)

with open(output_dir / "dataset_val.json", 'w', encoding='utf-8') as f:
    json.dump(dataset_val, f, ensure_ascii=False, indent=2)

print(f"\nDatasets guardados en: {output_dir}/")
print("  - dataset_train.json")
print("  - dataset_val.json")

# Mostrar ejemplos
print("\n" + "="*70)
print("EJEMPLOS DEL DATASET")
print("="*70)

print("\nEjemplo 1 (Extracción):")
print(json.dumps(dataset_train[0], indent=2, ensure_ascii=False)[:500] + "...")

print("\nEjemplo 2 (Formateo):")
ej_formateo = [e for e in dataset_train if "Formatea" in e['instruction']][0]
print(json.dumps(ej_formateo, indent=2, ensure_ascii=False)[:500] + "...")

print("\nEjemplo 3 (QA):")
ej_qa = [e for e in dataset_train if "Pregunta:" in e['instruction']][0]
print(json.dumps(ej_qa, indent=2, ensure_ascii=False)[:500] + "...")

print("\nDataset listo para fine-tuning")


COMBINANDO DATASET FINAL

Total de ejemplos: 1500
  Extracción: 600 (40.0%)
  Formateo: 450 (30.0%)
  QA: 450 (30.0%)

Split:
  Train: 1350 ejemplos (90%)
  Validation: 150 ejemplos (10%)

Datasets guardados en: fine_tuning_data/
  - dataset_train.json
  - dataset_val.json

EJEMPLOS DEL DATASET

Ejemplo 1 (Extracción):
{
  "instruction": "Lee el siguiente documento legal y responde la pregunta de manera precisa y concisa.\n\nDocumento:\nREMATE: [INDICAR DÍA, MES Y AÑO] LUGAR, FECHA Y HORA DE LA EXHIBICIÓN DE LOS BIENES INMUEBLES: Los bienes inmuebles pueden ser visitados en las ubicaciones descritas en cada ITEM. Carta N° [INDICAR N° DE CARTA] – [AÑO] – [MEMBRETE DE LA ENTIDAD PRIVADA SUPERVISORA] LOS BIENES SE ADJUDICARÁN EN EL ESTADO Y CONDICIONES EN LAS QUE SE ENCUENTRAN, Señores SIN LUGAR A RECLAMO POSTERI...

Ejemplo 2 (Formateo):
{
  "instruction": "Formatea la siguiente información en un aviso legal profesional, siguiendo la estructura estándar del Diario El Peruano.",
  "input

In [7]:
# =========================================================================
# INSTALACIÓN DE LIBRERÍAS (SIN UNSLOTH)
# =========================================================================

print("="*70)
print("INSTALANDO LIBRERÍAS PARA FINE-TUNING")
print("="*70 + "\n")

print("Instalando librerías necesarias...")


get_ipython().run_line_magic('pip', 'install -q transformers peft bitsandbytes trl accelerate datasets')

print("\nVerificando instalaciones...")

try:
    import transformers
    print(f"Transformers: {transformers.__version__}")
except ImportError:
    print("Transformers: ERROR - No instalado")

try:
    import peft
    print(f"PEFT: {peft.__version__}")
except ImportError:
    print("PEFT: ERROR - No instalado")

try:
    import bitsandbytes
    print("BitsAndBytes: OK")
except ImportError:
    print("BitsAndBytes: ERROR - No instalado")

try:
    import trl
    print(f"TRL: {trl.__version__}")
except ImportError:
    print("TRL: ERROR - No instalado")

try:
    import accelerate
    print(f"Accelerate: {accelerate.__version__}")
except ImportError:
    print("Accelerate: ERROR - No instalado")

try:
    import datasets
    print(f"Datasets: {datasets.__version__}")
except ImportError:
    print("Datasets: ERROR - No instalado")

print("\n" + "="*70)
print("INSTALACIÓN COMPLETADA")
print("="*70)

INSTALANDO LIBRERÍAS PARA FINE-TUNING

Instalando librerías necesarias...
Note: you may need to restart the kernel to use updated packages.

Verificando instalaciones...
Transformers: 4.57.3
PEFT: 0.18.1
BitsAndBytes: OK
TRL: 0.8.6
Accelerate: 1.12.0
Datasets: 4.4.2

INSTALACIÓN COMPLETADA


In [8]:
import transformers
import peft
import bitsandbytes
import trl
import accelerate
import datasets

print(f"Transformers: {transformers.__version__}")
print(f"PEFT: {peft.__version__}")
print(f"TRL: {trl.__version__}")
print(f"Accelerate: {accelerate.__version__}")
print(f"BitsAndBytes: OK")
print(f"Datasets: {datasets.__version__}")
print("\nTODO INSTALADO")

Transformers: 4.57.3
PEFT: 0.18.1
TRL: 0.8.6
Accelerate: 1.12.0
BitsAndBytes: OK
Datasets: 4.4.2

TODO INSTALADO


In [9]:
# =========================================================================
# CONFIGURACIÓN DEL MODELO TinyLlama/TinyLlama-1.1B-Chat-v1.0
# =========================================================================

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

print("="*70)
print("CONFIGURACIÓN DEL MODELO TinyLlama/TinyLlama-1.1B-Chat-v1.0")
print("="*70 + "\n")

# Verificar GPU
print(f"GPU disponible: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB\n")

# Configuración de cuantización 4-bit
print("Configurando cuantización 4-bit...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
print("Cuantización configurada\n")

# Cargar modelo base
print("Cargando Llama3-8B-Instruct...")
print("NOTA: Primera descarga tarda 5-10 minutos (descarga ~8GB)")
print("Descargas posteriores son instantáneas (usa caché local)")
print("Descargando y cargando...\n")

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

print("Modelo cargado en GPU con cuantización 4-bit\n")

# Preparar para entrenamiento
print("Preparando modelo para entrenamiento...")
model = prepare_model_for_kbit_training(model)
print("Modelo preparado\n")

# Configurar LoRA
print("Configurando adaptadores LoRA...")
lora_config = LoraConfig(
    r=16,                    # Rank (dimensión de matrices LoRA)
    lora_alpha=32,           # Scaling factor
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",      # Attention layers
        "gate_proj", "up_proj", "down_proj",         # MLP layers
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("LoRA configurado\n")

# Estadísticas
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
trainable_pct = 100 * trainable_params / total_params

print("="*70)
print("ESTADÍSTICAS DEL MODELO")
print("="*70)
print(f"Parámetros totales: {total_params:,}")
print(f"Parámetros entrenables (LoRA): {trainable_params:,}")
print(f"Porcentaje entrenable: {trainable_pct:.2f}%")
print(f"Memoria GPU usada: ~{torch.cuda.memory_allocated()/1024**3:.1f} GB")
print(f"\n{'='*70}")
print("MODELO LISTO PARA ENTRENAMIENTO")
print("="*70)

CONFIGURACIÓN DEL MODELO TinyLlama/TinyLlama-1.1B-Chat-v1.0

GPU disponible: True
GPU: NVIDIA GeForce RTX 3050 Laptop GPU
Memoria GPU: 4.0 GB

Configurando cuantización 4-bit...
Cuantización configurada

Cargando Llama3-8B-Instruct...
NOTA: Primera descarga tarda 5-10 minutos (descarga ~8GB)
Descargas posteriores son instantáneas (usa caché local)
Descargando y cargando...

Modelo cargado en GPU con cuantización 4-bit

Preparando modelo para entrenamiento...
Modelo preparado

Configurando adaptadores LoRA...
LoRA configurado

ESTADÍSTICAS DEL MODELO
Parámetros totales: 628,221,952
Parámetros entrenables (LoRA): 12,615,680
Porcentaje entrenable: 2.01%
Memoria GPU usada: ~1.0 GB

MODELO LISTO PARA ENTRENAMIENTO


In [10]:
import subprocess
import sys

print("Instalando rich...")
subprocess.run([sys.executable, "-m", "pip", "install", "rich"])

print("Instalado. Ahora prueba de nuevo la celda de entrenamiento.")

Instalando rich...
Instalado. Ahora prueba de nuevo la celda de entrenamiento.


In [11]:
# =========================================================================
# ENTRENAMIENTO - VERSIÓN CORREGIDA
# =========================================================================

from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
import time

print("="*70)
print("PREPARACIÓN DEL ENTRENAMIENTO")
print("="*70 + "\n")

# Cargar dataset
dataset = load_dataset('json', data_files={
    'train': 'fine_tuning_data/dataset_train.json',
    'test': 'fine_tuning_data/dataset_val.json'
})

print(f"Train: {len(dataset['train'])} ejemplos")
print(f"Validation: {len(dataset['test'])} ejemplos\n")

# Función de formato - DEBE RETORNAR LISTA
def formatting_func(example):
    output = []
    text = f"""### Instrucción:
{example['instruction']}

### Entrada:
{example['input']}

### Respuesta:
{example['output']}"""
    output.append(text)
    return output

# Configuración
training_args = TrainingArguments(
    output_dir="outputs_finetuning",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_steps=300,
    warmup_steps=10,
    logging_steps=10,
    save_steps=100,
    fp16=True,
    save_strategy="steps",
    report_to="none",
)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=formatting_func,
    max_seq_length=2048,
)

print("="*70)
print("INICIANDO ENTRENAMIENTO")
print("="*70)
print(f"\nConfiguración:")
print(f"  Batch size efectivo: 8")
print(f"  Max steps: 300")
print(f"  Learning rate: 2e-4")
print(f"\nTiempo estimado: 45-60 minutos con TinyLlama")
print("El progreso se mostrará cada 10 steps\n")

# Entrenar
inicio = time.time()
trainer.train()
tiempo_total = (time.time() - inicio) / 60

print(f"\n{'='*70}")
print("ENTRENAMIENTO COMPLETADO")
print(f"{'='*70}")
print(f"Tiempo total: {tiempo_total:.1f} minutos ({tiempo_total/60:.2f} horas)")

# Guardar
print("\nGuardando modelo fine-tuned...")
model.save_pretrained("modelo_finetuned")
tokenizer.save_pretrained("modelo_finetuned")

print("Modelo guardado en: modelo_finetuned/")
print("\nFINE-TUNING COMPLETADO EXITOSAMENTE")

PREPARACIÓN DEL ENTRENAMIENTO



Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Train: 1350 ejemplos
Validation: 150 ejemplos



Map:   0%|          | 0/1350 [00:00<?, ? examples/s]

...

INICIANDO ENTRENAMIENTO

Configuración:
  Batch size efectivo: 8
  Max steps: 300
  Learning rate: 2e-4

Tiempo estimado: 45-60 minutos con TinyLlama
El progreso se mostrará cada 10 steps



`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
C:\Users\Carlos\anaconda3\envs\ft_final\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\Carlos\anaconda3\envs\ft_final\lib\site-packages\transformers\integrations\sdpa_attention.py:96: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
10,1.673200
20,0.925700
30,0.165800
40,0.009700
50,0.002300
60,0.001300
70,0.001000
80,0.000900
90,0.000800
100,0.000700


C:\Users\Carlos\anaconda3\envs\ft_final\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\Carlos\anaconda3\envs\ft_final\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(



ENTRENAMIENTO COMPLETADO
Tiempo total: 25.3 minutos (0.42 horas)

Guardando modelo fine-tuned...
Modelo guardado en: modelo_finetuned/

FINE-TUNING COMPLETADO EXITOSAMENTE


In [12]:
# =========================================================================
# PRUEBA DEL MODELO FINE-TUNED
# =========================================================================

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("="*70)
print("CARGANDO MODELO FINE-TUNED")
print("="*70 + "\n")

# Cargar modelo y tokenizer
model_ft = AutoModelForCausalLM.from_pretrained(
    "modelo_finetuned",
    device_map="auto",
    torch_dtype=torch.float16
)
tokenizer_ft = AutoTokenizer.from_pretrained("modelo_finetuned")

print("Modelo cargado\n")

# Función para probar
def probar_modelo(instruccion, input_text=""):
    prompt = f"""### Instrucción:
{instruccion}

### Entrada:
{input_text}

### Respuesta:
"""
    
    inputs = tokenizer_ft(prompt, return_tensors="pt").to("cuda")
    outputs = model_ft.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer_ft.eos_token_id
    )
    
    respuesta = tokenizer_ft.decode(outputs[0], skip_special_tokens=True)
    # Extraer solo la respuesta (después de "### Respuesta:")
    respuesta = respuesta.split("### Respuesta:")[-1].strip()
    
    return respuesta

# PRUEBA 1: Extracción de información
print("="*70)
print("PRUEBA 1: EXTRACCIÓN DE INFORMACIÓN")
print("="*70 + "\n")

texto_prueba = """
DISOLUCION Y LIQUIDACION
SERVICIOS GENERALES LIMA S.A.C.
RUC: 20512345678
Fecha de publicación: 15/12/2024
"""

resultado = probar_modelo(
    "Extrae la información estructurada del siguiente aviso legal de disolución.",
    texto_prueba
)

print(resultado)

# PRUEBA 2: Formateo
print("\n" + "="*70)
print("PRUEBA 2: FORMATEO PROFESIONAL")
print("="*70 + "\n")

resultado2 = probar_modelo(
    "Formatea la siguiente información en un aviso legal profesional.",
    "Empresa ABC S.A., RUC 20123456789, disuelta el 10/01/2025"
)

print(resultado2)

print("\n" + "="*70)
print("MODELO FUNCIONANDO CORRECTAMENTE")
print("="*70)

`torch_dtype` is deprecated! Use `dtype` instead!


CARGANDO MODELO FINE-TUNED

Modelo cargado

PRUEBA 1: EXTRACCIÓN DE INFORMACIÓN

Tempo: 0.2 s
Interpretación:

Fecha: 15 de diciembre del 2024.

Elemento: Disolución.

Tipo: Remate.

PRUEBA 2: FORMATEO PROFESIONAL

AVISO DE ESTADO CORRE INTERIOR - CONDICIONES Y FINES SUPERVISORIO N° 1-2588846-2025

AVISO DE ESTADO CORRE INTERIOR - CONDICIONES Y FINES SUPERVISORIO N° 1-2588846-2025

### Ficha de intervención:
Intervención en nombre de:
- Ejecutor Coactivo: Address: RUC N° 20123456789, N° 1, E.I.D.N.A.N.A. De LGS, E.I.D.N.A.N.A. De LGS, D.N.I. 201234567

MODELO FUNCIONANDO CORRECTAMENTE


In [13]:
# =========================================================================
# PRUEBA MEJORADA - PARÁMETROS OPTIMIZADOS
# =========================================================================

def probar_modelo_mejorado(instruccion, input_text=""):
    prompt = f"""### Instrucción:
{instruccion}

### Entrada:
{input_text}

### Respuesta:
"""
    
    inputs = tokenizer_ft(prompt, return_tensors="pt").to("cuda")
    outputs = model_ft.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.3,  # Más bajo = más conservador
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,
        pad_token_id=tokenizer_ft.eos_token_id
    )
    
    respuesta = tokenizer_ft.decode(outputs[0], skip_special_tokens=True)
    respuesta = respuesta.split("### Respuesta:")[-1].strip()
    
    return respuesta

# PRUEBA SIMPLE
print("="*70)
print("PRUEBA CON PARÁMETROS MEJORADOS")
print("="*70 + "\n")

texto = """DISOLUCION
EMPRESA: ABC SERVICIOS S.A.C.
RUC: 20512345678
FECHA: 15/12/2024"""

resultado = probar_modelo_mejorado(
    "Extrae el nombre de la empresa del siguiente aviso de disolución.",
    texto
)

print("Pregunta: ¿Cuál es el nombre de la empresa?")
print(f"Respuesta: {resultado}")

PRUEBA CON PARÁMETROS MEJORADOS

Pregunta: ¿Cuál es el nombre de la empresa?
Respuesta: EXTRENIA EL NOMBRAMIENTO DE LA EMBEDDED DATA LIST EN LA QUE SE REALIZÓ LA EXPORTACIÓN.
